In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns    
%matplotlib inline

In [2]:
data_train_online = pd.DataFrame(pd.read_csv("./new_offline.csv",header = 0))

In [3]:
data_train_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 7 columns):
User_id          37 non-null int64
Merchant_id      37 non-null int64
Coupon_id        26 non-null float64
Discount_rate    26 non-null object
Distance         34 non-null float64
Date_received    26 non-null float64
Date             13 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 2.1+ KB


In [4]:
data_train_online.Date.isnull().value_counts()

True     24
False    13
Name: Date, dtype: int64

In [5]:
data_train_online.Coupon_id.isnull().value_counts()

False    26
True     11
Name: Coupon_id, dtype: int64

In [6]:
use_coupon = data_train_online[(data_train_online.Date.isnull().values == False)
                               & (data_train_online.Coupon_id.isnull().values == False)] #使用优惠券
use_coupon.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date
count,2.000000e+00,2.000000,2.000000,2.0,2.000000e+00,2.000000e+00
mean,1.276208e+06,1996.500000,9878.500000,0.0,2.016052e+07,2.016057e+07
std,2.307997e+05,898.732719,1820.799962,0.0,7.071068e-01,6.505382e+01
min,1.113008e+06,1361.000000,8591.000000,0.0,2.016052e+07,2.016052e+07
25%,1.194608e+06,1678.750000,9234.750000,0.0,2.016052e+07,2.016054e+07
50%,1.276208e+06,1996.500000,9878.500000,0.0,2.016052e+07,2.016057e+07
75%,1.357808e+06,2314.250000,10522.250000,0.0,2.016052e+07,2.016059e+07
max,1.439408e+06,2632.000000,11166.000000,0.0,2.016052e+07,2.016061e+07


In [7]:
not_use_coupon = data_train_online[(data_train_online.Date.isnull().values == True)
                               & (data_train_online.Coupon_id.isnull().values == False)] #领取未使用
not_use_coupon.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date
count,2.400000e+01,24.000000,24.000000,21.000000,2.400000e+01,0.0
mean,1.253950e+06,3631.333333,7700.791667,1.952381,2.016038e+07,NaN
std,9.030048e+05,2267.628289,4076.075190,3.456533,1.794424e+02,NaN
min,7.361100e+04,450.000000,1078.000000,0.000000,2.016011e+07,NaN
25%,3.211825e+05,1969.000000,4466.750000,0.000000,2.016021e+07,NaN
50%,1.276208e+06,3381.000000,8196.500000,0.000000,2.016042e+07,NaN
75%,1.980848e+06,4705.500000,11043.000000,2.000000,2.016052e+07,NaN
max,3.273056e+06,8390.000000,13490.000000,10.000000,2.016061e+07,NaN


In [8]:
not_have_coupon = data_train_online[(data_train_online.Date.isnull().values == False)
                               & (data_train_online.Coupon_id.isnull().values == True)]
not_have_coupon.describe() #未领取优惠券但购买了
not_have_coupon.count()

User_id          11
Merchant_id      11
Coupon_id         0
Discount_rate     0
Distance         11
Date_received     0
Date             11
dtype: int64

In [9]:
data_train_online.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0
7,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
8,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
9,2029232,450,1532.0,30:5,0.0,20160530.0,NaN


In [10]:
#添加列coupon_use，表示是否用券
data_train_online['coupon_use'] = data_train_online['Date'].notnull()&data_train_online['Coupon_id'].notnull()
data_train_online['coupon_use'] = data_train_online[['coupon_use']].astype(int)
#print(data_train_online['coupon_use'])

In [11]:
gb = data_train_online.groupby('User_id')

cols  = ['User_id','Merchant_id']
# 对data_train_online按照当前的User_id特征列进行分组，统计个数

#该用户共有几条记录
count=gb.size().reset_index(drop=False)
count.rename(columns={0:'total_count'}, inplace = True)
#print(count)
data_train_online=data_train_online.merge(count,how='left',on='User_id')

#该用户共领了几张券
count_m=gb['Coupon_id'].count().reset_index(drop=False)
count_m.rename(columns={'Coupon_id':'coupon_count'}, inplace = True)
print(count_m)
data_train_online=data_train_online.merge(count_m,how='left',on='User_id')

#该用户共购买几次
count_m=gb['Date'].count().reset_index(drop=False)
count_m.rename(columns={'Date':'buy_count'}, inplace = True)
print(count_m)
data_train_online=data_train_online.merge(count_m,how='left',on='User_id')



print(data_train_online)

    User_id  coupon_count
0     73611             1
1     94107             1
2    163606             1
3    196342             1
4    253750             2
5    343660             1
6    376492             1
7   1113008             5
8   1439408             5
9   1832624             1
10  1964720             1
11  2029232             3
12  2223968             1
13  2747744             1
14  3273056             1
15  4191584             0
    User_id  buy_count
0     73611          0
1     94107          0
2    163606          0
3    196342          1
4    253750          1
5    343660          0
6    376492          0
7   1113008          3
8   1439408          3
9   1832624          0
10  1964720          1
11  2029232          2
12  2223968          0
13  2747744          0
14  3273056          0
15  4191584          2
    User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0   1439408         2632        NaN           NaN       0.0            NaN   
1   1439408

In [12]:
#该用户共用券购买了几次
count_m=gb['coupon_use'].sum().reset_index(drop=False)
count_m.rename(columns={'coupon_use':'use_count'}, inplace = True)
print(count_m)
data_train_online=data_train_online.merge(count_m,how='left',on='User_id')

    User_id  use_count
0     73611          0
1     94107          0
2    163606          0
3    196342          0
4    253750          0
5    343660          0
6    376492          0
7   1113008          1
8   1439408          1
9   1832624          0
10  1964720          0
11  2029232          0
12  2223968          0
13  2747744          0
14  3273056          0
15  4191584          0


In [17]:
#未领取优惠券但购买了
data_train_online['coupon_not_use'] = data_train_online['Date'].notnull()&data_train_online['Coupon_id'].isnull()
data_train_online['coupon_not_use'] = data_train_online[['coupon_not_use']].astype(int)
#print(data_train_online['coupon_not_use'])

#领取优惠券但未购买
data_train_online['not_buy'] = data_train_online['Date'].isnull()&data_train_online['Coupon_id'].notnull()
data_train_online['not_buy'] = data_train_online[['not_buy']].astype(int)
print(data_train_online['not_buy'])

0     0
1     1
2     1
3     1
4     1
5     0
6     0
7     1
8     1
9     1
10    1
11    0
12    0
13    1
14    0
15    1
16    1
17    1
18    1
19    1
20    1
21    0
22    1
23    1
24    1
25    1
26    0
27    1
28    0
29    0
30    0
31    1
32    1
33    0
34    0
35    1
36    1
Name: not_buy, dtype: int64
